In [ ]:
import os
import random
import shutil

# 定义原始图片所在文件夹（请根据实际情况修改路径）
source_folders = {
    "flux": "/home/ps/sdbench/Shanshui/fluxlora_30_5",
    "sd3": "/home/ps/sdbench/Shanshui/SD3lora_40_5",
    "sdxl": "/home/ps/sdbench/Shanshui/sdxl_lora_5",
    "sd1.5": "/home/ps/sdbench/Shanshui/sdlora_50_5"
}

# 定义目标测试文件夹路径
dest_root = "/home/ps/zyp/evaluate/test10"

# 确保目标文件夹存在
os.makedirs(dest_root, exist_ok=True)

# 每个类别抽取的图片数量（即生成的子文件夹数）
num_samples = 1

# 允许的图片扩展名
valid_extensions = ('.png', '.jpg', '.jpeg', '.bmp', '.tif', '.tiff', '.webp')

for category, src_dir in source_folders.items():
    # 获取源文件夹中所有符合扩展名要求的图片
    images = [f for f in os.listdir(src_dir) if f.lower().endswith(valid_extensions)]
    
    if len(images) < num_samples:
        raise ValueError(f"文件夹 {src_dir} 中图片数量不足，至少需要 {num_samples} 张，当前只有 {len(images)} 张")
    
    # 随机打乱图片列表（确保不重复选择）
    random.shuffle(images)
    
    # 对于每个样本编号 1 到 num_samples，创建一个子文件夹并复制一张图片
    for i in range(1, num_samples + 1):
        # 构造新的子文件夹名称，如 flux_1, flux_2, … sd3_1, sd3_2, ...
        dest_folder = os.path.join(dest_root, f"{category}_{i}")
        os.makedirs(dest_folder, exist_ok=True)
        
        # 选择列表中的第 i-1 张图片
        src_image_path = os.path.join(src_dir, images[i - 1])
        dest_image_path = os.path.join(dest_folder, images[i - 1])
        
        # 复制图片到新的子文件夹
        shutil.copy2(src_image_path, dest_image_path)
        print(f"已复制 {src_image_path} 到 {dest_image_path}")



已复制 /home/ps/sdbench/Shanshui/fluxlora_30_5/20241212_105828.png 到 /home/ps/zyp/evaluate/test10/flux_1/20241212_105828.png
已复制 /home/ps/sdbench/Shanshui/SD3lora_40_5/20241212_192332.png 到 /home/ps/zyp/evaluate/test10/sd3_1/20241212_192332.png
已复制 /home/ps/sdbench/Shanshui/sdxl_lora_5/image_20241217-205423_000.png 到 /home/ps/zyp/evaluate/test10/sdxl_1/image_20241217-205423_000.png
已复制 /home/ps/sdbench/Shanshui/sdlora_50_5/im_20241212220927_000_2481609806.png 到 /home/ps/zyp/evaluate/test10/sd1.5_1/im_20241212220927_000_2481609806.png


## 最开始的改进CMMD

In [1]:
import os
print(os.getcwd())
os.chdir('/home/ps/sdbench')
print(os.getcwd())


/home/ps
/home/ps/sdbench


In [2]:
from evaluate import *
import os
import json
from tqdm import tqdm

def get_folders(folder_path):
    """获取指定文件夹下的所有子文件夹"""
    folder_paths = []
    for folder in os.listdir(folder_path):
        folder_path_full = os.path.join(folder_path, folder)
        if os.path.isdir(folder_path_full):
            folder_paths.append(folder_path_full)
    return folder_paths

def compute_and_save_scores(output_dir, result_fid_json):
    folder_paths = get_folders(output_dir)
    print(folder_paths)
    
    results = {}

    # 遍历每个文件夹，计算 CMMD 和 FID 分数
    print("Processing folders...")
    for folder_path in tqdm(folder_paths, desc="Processing folders"):
        folder_name = os.path.basename(folder_path)
        print(f"Processing folder: {folder_name}")
        
        # 计算 CMMD 和 FID 分数（针对整个文件夹）
        CMMD_score = calc_CMMD_Score('/home/ps/sdbench/Shanshui/Shanshui', folder_path)
        CMMD_plus_score = cmmd_single_image_against_ref('/home/ps/sdbench/Shanshui/Shanshui', folder_path)
        #fid_score = calc_Fid('/home/ps/sdbench/Shanshui/Shanshui', folder_path)
        
        # 将分数添加到结果字典中
        results[folder_name] = {
            'CMMD_score': CMMD_score,
            'CMMD_plus_score': CMMD_plus_score,
            #'FID_score': fid_score
        }

    # 保存结果到 JSON 文件
    with open(result_fid_json, 'w') as f:
        json.dump(results, f, indent=4)

    print(f"Results saved to {result_fid_json}")

if __name__ == "__main__":
    output_dir = "/home/ps/zyp/evaluate/test10"  # 目标文件夹路径
    result_fid_json = "/home/ps/zyp/evaluate/test10/result2.json"  # 保存结果的 JSON 文件路径
    compute_and_save_scores(output_dir, result_fid_json)


FileNotFoundError: [Errno 2] No such file or directory: '/home/ps/zyp/evaluate/test10'

In [50]:
import json

# 指定 JSON 文件路径
json_file_path = '/home/ps/zyp/evaluate/test10/result2.json'

# 从文件中读取 JSON 数据
with open(json_file_path, 'r', encoding='utf-8') as f:
    data = json.load(f)

# 按 CMMD_score 升序排序
sorted_by_cmmd = sorted(data.items(), key=lambda item: item[1]['CMMD_score'])

# 按 CMMD_plus_score 升序排序
sorted_by_cmmd_plus = sorted(data.items(), key=lambda item: item[1]['CMMD_plus_score'])

print("按 CMMD_score 升序排序：")
for folder, scores in sorted_by_cmmd:
    print(f"{folder}: {scores}")

print("\n按 CMMD_plus_score 升序排序：")
for folder, scores in sorted_by_cmmd_plus:
    print(f"{folder}: {scores}")

按 CMMD_score 升序排序：
sdxl_1: {'CMMD_score': 1.5823841094970703, 'CMMD_plus_score': 2.3500919342041016}
sd3_1: {'CMMD_score': 1.6552209854125977, 'CMMD_plus_score': 2.6106834411621094}
sd1.5_1: {'CMMD_score': 1.954793930053711, 'CMMD_plus_score': 1.9466876983642578}
flux_1: {'CMMD_score': 2.1022558212280273, 'CMMD_plus_score': 2.73895263671875}

按 CMMD_plus_score 升序排序：
sd1.5_1: {'CMMD_score': 1.954793930053711, 'CMMD_plus_score': 1.9466876983642578}
sdxl_1: {'CMMD_score': 1.5823841094970703, 'CMMD_plus_score': 2.3500919342041016}
sd3_1: {'CMMD_score': 1.6552209854125977, 'CMMD_plus_score': 2.6106834411621094}
flux_1: {'CMMD_score': 2.1022558212280273, 'CMMD_plus_score': 2.73895263671875}


## EMA+改进后的CMMD

In [1]:
import os
print(os.getcwd())
os.chdir('/home/ps/sdbench')
print(os.getcwd())

/home/ps
/home/ps/sdbench


In [3]:
import os
import json
import numpy as np
from collections import defaultdict
from tqdm import tqdm
from evaluate.CMMD_plus.embedding import ClipEmbeddingModel
from evaluate.CMMD_plus.io_util import compute_embeddings_for_dir
from evaluate.CMMD_plus.main import cumulative_cmmd

def get_folder_groups(folder_path):
    """按前缀（如 flux, sd3, sdxl）分组文件夹"""
    folder_groups = defaultdict(list)

    for folder in os.listdir(folder_path):
        folder_path_full = os.path.join(folder_path, folder)
        if os.path.isdir(folder_path_full):
            prefix = folder.split("_")[0]  # 获取前缀
            folder_groups[prefix].append(folder_path_full)

    return folder_groups

def compute_and_save_scores(output_dir, result_fid_json, ref_dir, alpha=0.9):
    folder_groups = get_folder_groups(output_dir)
    print("发现以下待处理的文件夹组:", folder_groups.keys())

    results = {}

    # **1. 预计算参考集的 embedding**
    print("\n计算参考图像集的 embedding（只计算一次）...")
    embedding_model = ClipEmbeddingModel()
    ref_embs = compute_embeddings_for_dir(
        img_dir=ref_dir,
        embedding_model=embedding_model,
        batch_size=32,
        max_count=-1
    ).astype("float32")

    print("参考集 embedding 计算完成！")

    # **2. 遍历测试文件夹组，计算 CMMD**
    print("\n开始计算前缀分组后的 CMMD 评分...")
    for prefix, folder_paths in tqdm(folder_groups.items(), desc="Processing folder groups"):
        print(f"\n处理前缀: {prefix}")

        # **获取所有图片路径**
        img_list = []
        for folder_path in folder_paths:
            img_list.extend([
                os.path.join(folder_path, f)
                for f in os.listdir(folder_path)
                if f.endswith(('.png', '.jpg', '.jpeg'))
            ])

        if not img_list:
            print(f"⚠️ 警告: 组 {prefix} 中没有找到图片，跳过！")
            continue

        # **计算 EMA 版本的 CMMD**
        CMMD_ema_score = cumulative_cmmd(
            ref_embs=ref_embs,  # 直接传入已计算好的参考集 embedding
            img_list=img_list,  # 该前缀的所有图片
            alpha=alpha,
            perturb_scale=1e-3
        )

        # **存入结果字典**
        results[prefix] = {
            'CMMD_ema_score': CMMD_ema_score
        }

    # **3. 保存计算结果到 JSON 文件**
    with open(result_fid_json, 'w') as f:
        json.dump(results, f, indent=4)

    print(f"\n所有计算完成 ✅ 结果已保存至: {result_fid_json}")

if __name__ == "__main__":
    output_dir = "/home/ps/zyp/evaluate/CMMD-plus/test-renwu"  # 目标文件夹路径
    result_fid_json = "/home/ps/zyp/evaluate/CMMD-plus/test-renwu/result_EMACMMD_1e-3.json"  # 结果 JSON
    ref_dir = "/home/ps/sdbench/train/datasets/Renwu"  # 参考图像目录

    compute_and_save_scores(output_dir, result_fid_json, ref_dir)


发现以下待处理的文件夹组: dict_keys(['sdxl', 'sd3', 'sd1.5', 'flux'])

计算参考图像集的 embedding（只计算一次）...
计算目录 /home/ps/sdbench/train/datasets/Renwu 中 33 张图片的embeddings.


2it [00:00,  2.47it/s]                                                                                                                                             


参考集 embedding 计算完成！

开始计算前缀分组后的 CMMD 评分...


Processing folder groups:   0%|                                                                                                              | 0/4 [00:00<?, ?it/s]


处理前缀: sdxl


...


处理前缀: sd1.5


Processing folder groups:  75%|███████████████████████████████████████████████████████████████████████████▊                         | 3/4 [05:11<01:43, 103.79s/it]


处理前缀: flux


Processing folder groups: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [06:53<00:00, 103.41s/it]


所有计算完成 ✅ 结果已保存至: /home/ps/zyp/evaluate/CMMD-plus/test-renwu/result_EMACMMD_1e-3.json


In [4]:
import json

# 读取 JSON 文件
file_path = "/home/ps/zyp/evaluate/CMMD-plus/test-renwu/result_EMACMMD_1e-3.json"

with open(file_path, "r") as f:
    data = json.load(f)

# 按 `CMMD_ema_score` 进行升序排序
sorted_data = sorted(data.items(), key=lambda x: x[1]["CMMD_ema_score"])

# 打印排序后的结果
for folder, scores in sorted_data:
    print(f"{folder}: {scores['CMMD_ema_score']}")


flux: 4.027120633054732
sdxl: 4.169022790382504
sd3: 4.2550903761523955
sd1.5: 5.353388650109411


# ConvNext v2实验